<a href="https://colab.research.google.com/github/tlsgptj/MLP/blob/MLP3/%ED%92%8D%EB%A0%A5%EB%B0%9C%EC%A0%84_%EB%B0%9C%EC%A0%84%EB%9F%89_%EC%98%88%EC%B8%A1(MLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#풍력발전 데이터 업로드
import pandas as pd

In [ ]:
df = pd.read_csv("풍력발전.csv", encoding="cp949")

In [ ]:
df

In [ ]:
df = df.drop(columns = ["발전기코드", "발전기명", "설비용량(MW)", "풍향(16방위)"])

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#matplotlib 라이브러리를 활용하여 발전량을 파악해본다.
#파악하기 전, 풍력발전 데이터를 시간이 아닌 일별 평균으로 만들어준다.
# 데이터를 묶을 크기 설정
chunk_size = 24  # 묶을 크기를 원하는 값으로 변경하세요

# 묶은 데이터의 평균 계산
df_averge = df.groupby(df.index // chunk_size).mean()

# 결과 출력
print(df_averge)

In [ ]:
df_averge
#이런식으로 1시간 단위였던 발전량과 풍속, 풍향이 1일 평균으로 되었다. 여기서 시간은 필요없으니 삭제
df_averge = df_averge.drop(columns = ["시간"])

In [ ]:
df_averge

In [ ]:
#그렇다면 이제 거래일, 발전량, 풍속, 풍향을 비교하여 발전량이 어떻게 변화하는지 알아보겠다.
x=df_averge["풍속(m/s)"]
y1=df_averge["발전량(MWh)"]

# 그래프 그리기
plt.scatter(x, y1, label='generate')

# 그래프 제목, 축 레이블 설정
plt.title('Comparison of power generation according to wind speed')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.xticks(pd.date_range(start=x.min(), end=x.max(), freq='1H'), rotation=45)
plt.show()

#풍속에 따른 발전량을 비교해보니 풍속이 증가하면 발전량도 증가하는 1차함수 느낌의 그래프가 만들어졌다.

In [ ]:
x=df_averge["풍향"]
y1=df_averge["발전량(MWh)"]

# 그래프 그리기
plt.scatter(x, y1, label='generate')

# 그래프 제목, 축 레이블 설정
plt.title('Comparison of power generation according to wind Derection')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.xticks(pd.date_range(start=x.min(), end=x.max(), freq='1H'), rotation=45)
plt.show()

#그래프가 중구난방인것을 보니 풍향에 따른 발전량은 서로 관련이 없어 보인다.

In [ ]:
#거래일 정규화
df_averge["거래일"] = df_averge["거래일"].astype(float)/abs(df_averge["거래일"].astype(float).max())
df_averge["거래일"].head()

In [ ]:
#발전량 정규화
df_averge["발전량(MWh)"] = df_averge["발전량(MWh)"].astype(float)/abs(df_averge["발전량(MWh)"].astype(float).max())
df_averge["발전량(MWh)"].head()

In [ ]:
#풍속 정규화
df_averge["풍속(m/s)"] = df_averge["풍속(m/s)"].astype(float)/abs(df_averge["풍속(m/s)"].astype(float).max())
df_averge["풍속(m/s)"].head()

In [ ]:
#풍향 정규화
df_averge["풍향"] = df_averge["풍향"].astype(float)/abs(df_averge["풍향"].astype(float).max())
df_averge["풍향"].head()

In [ ]:
x=df_averge["발전량(MWh)"]
y1=df_averge["풍향"]
y2=df_averge["풍속(m/s)"]
# 그래프 그리기
plt.scatter(x, y1, label='wind direction')
plt.scatter(x, y2, label='wind speed')

# 그래프 제목, 축 레이블 설정
plt.title('Comparison of power generation according to wind speed, wind direction')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.xticks(pd.date_range(start=x.min(), end=x.max(), freq='1H'), rotation=45)
plt.show()

#거래일과 발전량은 정규화를 시켜주었다.

In [ ]:
#여러번의 비교를 통해 풍속에 따른 발전량을 예측해보는 프로토타입 앱을 제작해보기로 했다.
#예측은 MLP의 여러가지의 것들로 사용해보았다.
#첫 번째는 가장 많이 사용되는 adam을 사용했다.
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Test loss: 0.03709577023983002
#1/1 [==============================] - 0s 159ms/step
#Next value prediction: [[0.42929763]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Test loss: 0.03861237317323685
#1/1 [==============================] - 0s 103ms/step
#Next value prediction: [[0.38661167]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adagrad', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adagrad
#Test loss: 0.03483494371175766
#1/1 [==============================] - 0s 65ms/step
#Next value prediction: [[0.2286938]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='AdaDelta', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adadelta
#Test loss: 0.05127893388271332
#1/1 [==============================] - 0s 86ms/step
#Next value prediction: [[0.23875901]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Nadam', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Nadam
#Test loss: 0.04514951631426811
#1/1 [==============================] - 0s 62ms/step
#Next value prediction: [[0.27894822]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='SGD', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#SGD
#Test loss: 0.03544986620545387
#WARNING:tensorflow:6 out of the last 6 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f95d7118280> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
#1/1 [==============================] - 0s 63ms/step
#Next value prediction: [[0.313902]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#표로 만들어 가장 오차가 적은 것을 분석해보았다.
#분석내용 표로 도출
data = {
    'Optimizer': ['adam', 'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam'],
    '오차': [0.03709577023983002, 0.03544986620545387, 0.03861237317323685, 0.03483494371175766, 0.05127893388271332, 0.034454286098480225, 0.04514951631426811],
    'Epoch': [150, 150, 150, 150, 150, 150, 150],
    'Next value prediction': [0.23875901, 0.313902, 0.38661167, 0.2286938, 0.23875901, 0.35163516, 0.27894822, ]
}

df_7 = pd.DataFrame(data, dtype=float)

In [ ]:
df_7

In [ ]:
#이것을 그래프로 시각화 해보겠다.
x=df_7["Optimizer"]
y1=df_7["오차"]

# 그래프 그리기
plt.plot(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.show()

#오차를 살펴보니, adadelta가 가장 오차가 높았고, Adamax가 가장 오차가 적었다.
#Adamax를 통해 Epoch등 비교를 해보기로 했다.

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=200, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
model.save('mlp_model.h5')

In [ ]:
#epochs=200, batch_size=32, keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    #keras.layers.Dense(24
#Test loss: 0.037048276513814926
#1/1 [==============================] - 0s 64ms/step
#Next value prediction: [[0.50455594]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=200, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    #keras.layers.Dense(48, activation='relu'),
#epochs=200, batch_size=32
#Test loss: 0.042381808161735535
#1/1 [==============================] - 0s 59ms/step
#Next value prediction: [[0.48523682]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    #keras.layers.Dense(48,
#epochs=500, batch_size=64
#Test loss: 0.04714444652199745
#1/1 [==============================] - 0s 70ms/step
#Next value prediction: [[0.42497924]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
 #keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    #keras.layers.Dense(24,
#epochs=500, batch_size=64)
#Test loss: 0.04599790275096893
#1/1 [==============================] - 0s 58ms/step
#Next value prediction: [[0.54427695]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=1000, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
 # keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
  #  keras.layers.Dense(48, activation='relu'),
  #epochs=1000, batch_size=32)
#Test loss: 0.08605163544416428
#1/1 [==============================] - 0s 86ms/step
#Next value prediction: [[0.2904066]]

In [ ]:
data = df_averge["풍속(m/s)"]
target = df_averge["발전량(MWh)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
import numpy as np

window_size = 10  # 입력 시퀀스의 길이
x = []
y = []

for i in range(len(df) - window_size):
    # 데이터 프레임의 인덱스 범위를 벗어나면 스킵
    if i+window_size >= len(df_1):
        continue

    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])

x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=1000, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    #keras.layers.Dense(24,
#epochs=1000, batch_size=64
#Test loss: 0.059407636523246765
#1/1 [==============================] - 0s 95ms/step
#Next value prediction: [[0.41624594]]

In [ ]:
#표로 정리
data = {
    '번호': [1, 2, 3, 4, 5, 6],
    'Keras의 Dense 층': ['48/24', '96/48', '96/48', '48/24', '96/48', '48/24'],
    '오차': [0.037048276513814926, 0.042381808161735535, 0.04714444652199745, 0.04599790275096893, 0.08605163544416428, 0.059407636523246765],
    'Epoch': [200, 200, 500, 500, 1000, 1000],
    'Next value prediction': [0.50455594, 0.48523682, 0.42497924, 0.54427695, 0.2904066, 0.41624594],
    'Batch_size': [32, 32, 64, 64, 32, 64]
}

df_8 = pd.DataFrame(data, dtype=float)

In [ ]:
df_8

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x = df_8["번호"]
y1 = df_8["오차"]

# 그래프 그리기
plt.plot(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.ylabel('Y-axis')

# 그래프 보이기
plt.show()

# 가장 오차가 적음 : 48/24	0.037048	200.0	0.504556	32.0

In [ ]:
pip install streamlit

In [ ]:
#이것으로 streamlit으로 구축해보겠다.
import streamlit as st
import tensorflow as tf
from tensorflow import keras

# MLP 모델 불러오기
model = keras.models.load_model('mlp_model.h5')

# Streamlit 앱 설정
st.title("풍력 발전량 예측")

# 풍속 입력 필드
speed = st.number_input("풍속(m/s)", min_value=0.0)

# 예측 함수
def predict_power(speed):
    power = model.predict([[speed]])[0][0]
    return power

# 예측 버튼
if st.button("예측"):
    result = predict_power(speed)
    st.write(f"풍력 발전량 예측: {result:.2f} MWh")

In [ ]:
pip install pyngrok

In [ ]:
#ngrok으로 url표현
from pyngrok import ngrok

# ngrok 실행
ngrok_tunnel = ngrok.connect(8501)

# 할당된 URL 확인
public_url = ngrok_tunnel.public_url
print(f"ngrok URL: {public_url}")

# 앱 실행 후 종료 시 ngrok 연결 해제
# ...

# ngrok 연결 해제
ngrok.kill()

In [ ]:
!nohup streamlit run app.py --server port 80 &